In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import json

# -----------------------------
# CONFIG
# -----------------------------
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 8
DATASET_PATH = "model_1_dataset"

# -----------------------------
# LOAD DATASETS
# -----------------------------
train_ds = tf.keras.utils.image_dataset_from_directory(
    f"{DATASET_PATH}/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

valid_ds = tf.keras.utils.image_dataset_from_directory(
    f"{DATASET_PATH}/valid",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    f"{DATASET_PATH}/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)

print("Classes:", class_names)
print("Number of classes:", num_classes)

# -----------------------------
# SAVE CLASS NAMES (IMPORTANT)
# -----------------------------
with open("crop_class_names.json", "w") as f:
    json.dump(class_names, f)

# -----------------------------
# DATA PREPROCESSING
# -----------------------------
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(ds):
    return ds.map(
        lambda x, y: (preprocess_input(x), y),
        num_parallel_calls=AUTOTUNE
    ).prefetch(AUTOTUNE)

train_ds = preprocess(train_ds)
valid_ds = preprocess(valid_ds)
test_ds  = preprocess(test_ds)

# -----------------------------
# DATA AUGMENTATION (SAFE)
# -----------------------------
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# -----------------------------
# MODEL DEFINITION
# -----------------------------
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # Transfer Learning

model = models.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

# -----------------------------
# COMPILE MODEL
# -----------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# -----------------------------
# TRAIN MODEL
# -----------------------------
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=EPOCHS
)

# -----------------------------
# TEST EVALUATION
# -----------------------------
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# -----------------------------
# SAVE MODEL (SAFE FORMAT)
# -----------------------------
model.save("crop_identification_model.keras")

print("✅ Model training and saving completed successfully.")


Found 1258 files belonging to 4 classes.
Found 359 files belonging to 4 classes.
Found 182 files belonging to 4 classes.
Classes: ['Cotton', 'Maize', 'Pigeon Pea', 'Soybean']
Number of classes: 4


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 54s 935ms/step - accuracy: 0.7758 - loss: 0.6515 - val_accuracy: 0.9359 - val_loss: 0.2239
Epoch 2/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.9444 - loss: 0.1830 - val_accuracy: 0.9749 - val_loss: 0.1053
Epoch 3/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.9825 - loss: 0.0966 - val_accuracy: 0.9861 - val_loss: 0.0700
Epoch 4/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.9841 - loss: 0.0693 - val_accuracy: 0.9889 - val_loss: 0.0513
Epoch 5/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 88s 1s/step - accuracy: 0.9936 - loss: 0.0420 - val_accuracy: 0.9972 - val_loss: 0.0442
Epoch 6/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.9960 - loss: 0.0325 - val_accuracy: 0.9944 - val_loss: 0.0393
Epoch 7/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 32s 776ms/step - accuracy: 0.9976 - loss: 0.0237 - val_accuracy: 0.9972 - val_loss: 0.0331
Epoch 8/8
40/40 ━━━━━━━━━━━━━━━━━━━━ 31s 767ms/step - accuracy: 0.9921 - loss: 0.0266 - val_accuracy: 0.9972 - val_loss

In [9]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load model & classes
model = tf.keras.models.load_model("crop_identification_model.keras")

with open("crop_class_names.json") as f:
    class_names = json.load(f)

IMG_SIZE = (224, 224)

def predict_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    preds = model.predict(img_array)[0]

    confidence = np.max(preds) * 100
    index = np.argmax(preds)
    label = class_names[index]

    # Entropy check
    entropy = -np.sum(preds * np.log(preds + 1e-10))

    if label == "Unknown" or confidence < 70 or entropy > 1.2:
        return "Unknown Crop", confidence

    return label, confidence


crop, conf = predict_image("0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417.JPG")
print(f"Prediction: {crop} ({conf:.2f}%)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction: Unknown Crop (34.51%)
